In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import collections

In [2]:
%%time
raw_data = pd.read_csv(r"C:\Users\wwang\Downloads\hotel_review_in_europe\Hotel_Reviews.csv")
raw_data = raw_data.drop_duplicates()[:10000]
print(len(raw_data))

10000
Wall time: 2.75 s


In [3]:
raw_data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [44]:
from sklearn import datasets
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from nltk.stem import PorterStemmer
from nltk import ngrams
from nltk.corpus import stopwords
en_stop = set(stopwords.words('english'))

In [22]:
individual_review_drop_set = set(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date', 'Average_Score',\
                                  'Hotel_Name', 'Reviewer_Nationality', 'Review_Total_Negative_Word_Counts', \
                                  'Total_Number_of_Reviews', 'Review_Total_Positive_Word_Counts', \
                                  'Total_Number_of_Reviews_Reviewer_Has_Given', 'days_since_review', 'lat', 'lng'])
individual_review_raw_data = raw_data.drop(list(individual_review_drop_set), 1)

# Remove stopwords
individual_review_raw_data['Positive_Review'] = individual_review_raw_data['Positive_Review'].apply(lambda x: ' '.join([w for w in x.strip().lower().split() if w not in en_stop]))
individual_review_raw_data['Negative_Review'] = individual_review_raw_data['Negative_Review'].apply(lambda x: ' '.join([w for w in x.strip().lower().split() if w not in en_stop]))

# Stemming
ps = PorterStemmer()
individual_review_raw_data['Positive_Review'] = individual_review_raw_data['Positive_Review'].apply(lambda x: ' '.join([ps.stem(y) for y in x.split()]))
individual_review_raw_data['Negative_Review'] = individual_review_raw_data['Negative_Review'].apply(lambda x: ' '.join([ps.stem(y) for y in x.split()]))

X = individual_review_raw_data.drop(["Reviewer_Score"], 1)
y = individual_review_raw_data["Reviewer_Score"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=310)
X_train.head()

D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


,Negative_Review,Positive_Review,Tags
208,neg nois build work go time awar book stay hug...,staff extrem polit help honestli fault room cl...,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
1682,noth,good locat help staff room spaciou well worth ...,"[' Leisure trip ', ' Group ', ' Superior King ..."
204,neg,good locat next tram go major revamp nois prob...,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
150,sofa need bar better breakfast option water pr...,locat great us close sister alway someon desk ...,"[' Leisure trip ', ' Group ', ' Duplex Double ..."
6761,room took warm put air con high temp heat,staff friendli help,"[' Leisure trip ', ' Couple ', ' Superior Doub..."


In [23]:
X_test.head()

,Negative_Review,Positive_Review,Tags
8374,notic lack face cloth bathroom guess british c...,breakfast buffet excel staff attent staff over...,"[' Leisure trip ', ' Solo traveler ', ' Deluxe..."
9011,room suppos superior doubl room window open pr...,breafast good person also good cater peopl eat...,"[' Couple ', ' Superior Double Room ', ' Staye..."
1144,neg,great staff arriv delay wait readi 3am upgrad ...,"[' Leisure trip ', ' Solo traveler ', ' Deluxe..."
9409,room price bit steep even fine includ breakfas...,hotel beauti amaz antiqu recept hall entir exp...,"[' Leisure trip ', ' Couple ', ' Superior Doub..."
6250,neg,excel locat 5 min walk london eye well maintai...,"[' Leisure trip ', ' Family with young childre..."


In [25]:
%%time
# NGram
ngram_cnt = collections.Counter(
    [a for a in ngrams(" | ".join(X_train['Positive_Review']).split(), 1)] + \
    [a for a in ngrams(" | ".join(X_train['Positive_Review']).split(), 2)] + \
    [a for a in ngrams(" | ".join(X_train['Positive_Review']).split(), 3)] + \
    [a for a in ngrams(" | ".join(X_train['Negative_Review']).split(), 1)] + \
    [a for a in ngrams(" | ".join(X_train['Negative_Review']).split(), 2)] + \
    [a for a in ngrams(" | ".join(X_train['Negative_Review']).split(), 3)])
print([(k, c) for k, c in ngram_cnt.most_common(10) if '|' not in k])
features = [" ".join(k) for k, _ in ngram_cnt.most_common(200) if '|' not in k]

[(('room',), 7747), (('staff',), 3858), (('hotel',), 3413), (('locat',), 3349), (('breakfast',), 2054), (('good',), 2007), (('neg',), 1779), (('bed',), 1768)]
Wall time: 337 ms


In [26]:
%%time
for f in features:
    X_train["feature_p_" + "__".join(f.split())] = X_train['Positive_Review'].apply(lambda x: f in x)
    X_train["feature_n_" + "__".join(f.split())] = X_train['Negative_Review'].apply(lambda x: f in x)
    
    X_test["feature_p_" + "__".join(f.split())] = X_test['Positive_Review'].apply(lambda x: f in x)
    X_test["feature_n_" + "__".join(f.split())] = X_test['Negative_Review'].apply(lambda x: f in x)
X_train.head()

D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

Wall time: 798 ms


,Negative_Review,Positive_Review,Tags,feature_p_room,feature_n_room,feature_p_staff,feature_n_staff,feature_p_hotel,feature_n_hotel,feature_p_locat,...,feature_p_bed__comfort,feature_n_bed__comfort,feature_p_polit,feature_n_polit,feature_p_howev,feature_n_howev,feature_p_tea,feature_n_tea,feature_p_excel__locat,feature_n_excel__locat
208,neg nois build work go time awar book stay hug...,staff extrem polit help honestli fault room cl...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",True,False,True,False,False,False,True,...,False,False,True,False,False,False,False,False,False,False
1682,noth,good locat help staff room spaciou well worth ...,"[' Leisure trip ', ' Group ', ' Superior King ...",True,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
204,neg,good locat next tram go major revamp nois prob...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
150,sofa need bar better breakfast option water pr...,locat great us close sister alway someon desk ...,"[' Leisure trip ', ' Group ', ' Duplex Double ...",True,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
6761,room took warm put air con high temp heat,staff friendli help,"[' Leisure trip ', ' Couple ', ' Superior Doub...",False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
X_test.head()

,Negative_Review,Positive_Review,Tags,feature_p_room,feature_n_room,feature_p_staff,feature_n_staff,feature_p_hotel,feature_n_hotel,feature_p_locat,...,feature_p_bed__comfort,feature_n_bed__comfort,feature_p_polit,feature_n_polit,feature_p_howev,feature_n_howev,feature_p_tea,feature_n_tea,feature_p_excel__locat,feature_n_excel__locat
8374,notic lack face cloth bathroom guess british c...,breakfast buffet excel staff attent staff over...,"[' Leisure trip ', ' Solo traveler ', ' Deluxe...",False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9011,room suppos superior doubl room window open pr...,breafast good person also good cater peopl eat...,"[' Couple ', ' Superior Double Room ', ' Staye...",False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1144,neg,great staff arriv delay wait readi 3am upgrad ...,"[' Leisure trip ', ' Solo traveler ', ' Deluxe...",True,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
9409,room price bit steep even fine includ breakfas...,hotel beauti amaz antiqu recept hall entir exp...,"[' Leisure trip ', ' Couple ', ' Superior Doub...",True,True,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
6250,neg,excel locat 5 min walk london eye well maintai...,"[' Leisure trip ', ' Family with young childre...",False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False


In [30]:
X_train = X_train.drop(["Positive_Review", "Negative_Review", "Tags"], 1)
X_train.head()

X_test = X_test.drop(["Positive_Review", "Negative_Review", "Tags"], 1)
X_test.head()

D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
D:\programs\Anaconda3\envs\cs410\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,feature_p_room,feature_n_room,feature_p_staff,feature_n_staff,feature_p_hotel,feature_n_hotel,feature_p_locat,feature_n_locat,feature_p_breakfast,feature_n_breakfast,...,feature_p_bed__comfort,feature_n_bed__comfort,feature_p_polit,feature_n_polit,feature_p_howev,feature_n_howev,feature_p_tea,feature_n_tea,feature_p_excel__locat,feature_n_excel__locat
8374,False,True,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
9011,False,True,False,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1144,True,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9409,True,True,True,False,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
6250,False,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [47]:
rf_model = RandomForestRegressor(n_estimators=500, max_depth=25, n_jobs=-1, min_samples_split=10, min_samples_leaf=2)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=25, min_samples_leaf=2, min_samples_split=10,
                      n_estimators=500, n_jobs=-1)

In [48]:
import random
r = []
for i in range(len(y_test)):
    n = random.randint(1,10)
    r.append(n)
print("baseline msr:", mean_squared_error(r, y_test.astype('float')))
print("model msr:", mean_squared_error(rf_model.predict(X_test), y_test.astype('float')))

baseline msr: 18.599230000000002
model msr: 1.6041460983597615


In [49]:
rf_model2 = RandomForestRegressor(n_estimators=5000, max_depth=15, n_jobs=-1, min_samples_split=5, min_samples_leaf=2)
rf_model2.fit(X_train, y_train)
print("model msr:", mean_squared_error(rf_model2.predict(X_test), y_test.astype('float')))

model msr: 1.626617954000592
